## Power calcuations

In this notebook we perform the power calculations that appear in our pre-analysis plan.

In [ ]:
import io

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.stats import api as sms

%matplotlib inline

In [ ]:
# Here we set out how many hypotheses we are testing (NUM_HYPOTHESES),
# the desired false positive rate of the study (ALPHA) and the
# desired True Negative Rate (POWER). The number of hypotheses is used
# for a Bonferoni correction.

NUM_HYPOTHESES = 8
ALPHA = 0.05
POWER = 0.8

### Binary outcomes

In this section we focus on binary outcomes. We produce graphs of what effects will be detectable if we have three different sample sizes (called `Upper`, `Medium`, and `Lower`) and different base rates (spanning `Base Lower` and `Base Upper`).

In [ ]:
binary_df = pd.read_csv(
    io.StringIO(
        """
Description,Upper,Medium,Lower,Base Lower,Base Upper
PCP,16665,5400.0,2400,0.07,0.12
ED,16665,5400.0,2400,0.88,0.93
Adherence,1680,544.3744374437443,240,0.5,0.55
Ambulance,25200,8165.616561656165,3600,0.9,0.96
Misuse,25200,8165.616561656165,3600,0.13,0.17
"""
    )
)

In [ ]:
# All effect sizes are computed in terms of Cohen's h. This inverts Cohen's h to give a range of proportions


def hinv(h, p):
    phi = 2 * np.arcsin(np.sqrt(p))
    p1 = np.sin((h - phi) / 2) ** 2
    p2 = np.sin((h + phi) / 2) ** 2
    return tuple(sorted((p1, p2)))

In [ ]:
p = sms.NormalIndPower()

for _, row in binary_df.iterrows():
    xrange = np.linspace(row["Base Lower"], row["Base Upper"], 30)
    for sample_size_name, color in zip(
        ["Lower", "Medium", "Upper"], ["green", "orange", "blue"]
    ):
        sample_size = row[sample_size_name]
        effect_size = p.solve_power(
            nobs1=sample_size / 2, alpha=ALPHA / NUM_HYPOTHESES, power=POWER
        )
        ys = [hinv(effect_size, x) for x in xrange]
        plt.fill_between(
            xrange,
            [y[0] for y in ys],
            [y[1] for y in ys],
            label=f"{int(sample_size)}",
            facecolor=color,
        )
    plt.xlabel("Base Rate")
    plt.ylabel("Undetectable Range")
    plt.legend(loc="upper left")
    plt.show();

### Continuous outcomes

In this section we focus on continuous outcomes. We produce graphs of what effects will be detectable if we have three different sample sizes (called `Upper`, `Medium`, and `Lower`) and different mean base rates (spanning `Base Mean Lower` and `Base Mean Upper`) for a fixed standard deviation in our population (`Standard Deviation`).

In [ ]:
continuous_df = pd.read_csv(
    io.StringIO(
        """
Description,Upper,Medium,Lower,Base Mean Lower,Base Mean Upper,Standard Deviation
Hardship,25200,8165.616561656165,3600,400,500,225
Savings,25200,8165.616561656165,3600,-1,1,225
Expenditures,25200,8165.616561656165,3600,8000,10000,5000
"""
    )
)

In [ ]:
# All effect sizes are computed in terms of Cohen's d. This inverts Cohen's d to give a range of proportions


def dinv(d, mu, s):
    return (mu - d * s, mu + d * s)

In [ ]:
p = sms.TTestIndPower()

for _, row in continuous_df.iterrows():
    xrange = np.linspace(row["Base Mean Lower"], row["Base Mean Upper"], 30)
    for sample_size_name, color in zip(
        ["Lower", "Medium", "Upper"], ["green", "orange", "blue"]
    ):
        sample_size = row[sample_size_name]
        effect_size = p.solve_power(
            nobs1=sample_size / 2, alpha=ALPHA / NUM_HYPOTHESES, power=POWER
        )
        ys = [dinv(effect_size, x, row["Standard Deviation"]) for x in xrange]
        plt.fill_between(
            xrange,
            [y[0] for y in ys],
            [y[1] for y in ys],
            label=f"{int(sample_size)}",
            facecolor=color,
        )
    plt.xlabel("Base Rate")
    plt.ylabel("Undetectable Range")
    plt.legend(loc="upper left")
    plt.show();